In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tqdm.notebook import tqdm

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

In [17]:
IMAGE_SIZE = 224 # Image size of resize when applying transforms.
BATCH_SIZE = 64
NUM_WORKERS = 4 # Number of parallel processes for data preparation.
SUBMISSION = 'submission.csv'
DATA_DIR = 'test'
MODEL_DIR = 'model.pth'

In [18]:
import torchvision.models as models
import torch.nn as nn

def build_model(pretrained=True, fine_tune=True, num_classes=11):
    if pretrained:
        print('[INFO]: Loading pre-trained weights')
    else:
        print('[INFO]: Not loading pre-trained weights')
    model = models.efficientnet_b0(pretrained=pretrained)
    if fine_tune:
        print('[INFO]: Fine-tuning all layers...')
        for params in model.parameters():
            params.requires_grad = True
    elif not fine_tune:
        print('[INFO]: Freezing hidden layers...')
        for params in model.parameters():
            params.requires_grad = False
            
    num_features = model.classifier[-1].in_features
    # Change the final classification head.
    model.classifier[-1] = nn.Sequential(
      nn.Dropout(p=0.5),
      nn.Linear(num_features,num_classes),
      nn.Softmax(dim=1) 
    )
    return model

In [19]:
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
            )
])

In [20]:
import os
from PIL import Image
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_names = os.listdir(root_dir)
    
    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_names[idx])
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)
        img_name = self.image_names[idx]
        uid = int(img_name[len("test"):img_name.find(".")])
#         uid = int(img_name[:img_name.find(".")])
        return image, uid

In [21]:
dataset_test = CustomDataset(
    DATA_DIR, 
    transform=test_transform
)

In [22]:
test_loader = DataLoader(
        dataset_test, batch_size=BATCH_SIZE, 
        shuffle=False
)

In [23]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [24]:
model = build_model().to(device)

checkpoint = torch.load(MODEL_DIR, map_location=device)
print('Loading trained model weights...')
model.load_state_dict(checkpoint)

[INFO]: Loading pre-trained weights
[INFO]: Fine-tuning all layers...


C:\Users\OSM\.conda\envs\data-midterm\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\OSM\.conda\envs\data-midterm\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading trained model weights...


<All keys matched successfully>

In [25]:
import pandas as pd

In [26]:
ids = list()
classes = list()
threshold = 0.5

with torch.no_grad():
    for ( inputs, labels ) in tqdm(test_loader):
        model.eval()
        
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        scores, predicted = torch.max(outputs.data, 1)
        
#         for i in range(len(scores)):
#             if scores[i].item() < threshold:
#                 print(f"Image {labels[i].item()} is unclassified with score:", scores[i].item())
#                 predicted[i] = 8 # assign spam class to the prediction
            
        classes.extend(predicted.cpu().numpy())
        ids.extend(labels.cpu().numpy())

should_be = []
for i in range(len(ids)):
    target = 1
    # anything in `test` should not be spam  except ids > 10000
    if ids[i] > 10000 :
        target = 0
        
    if classes[i] == 8 :
        classes[i] = 0
    else :
        classes[i] = 1
    should_be.append(target)

        
df = pd.DataFrame({'ID': ids, 'class': classes})
df.to_csv(SUBMISSION, index=False)

  0%|          | 0/74 [00:00<?, ?it/s]

In [27]:
from sklearn.metrics import confusion_matrix
# Calculate confusion matrix
cm = confusion_matrix(should_be, classes)

print(cm)

[[  23    5]
 [ 106 4542]]
